In [1]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
import joblib
import os

In [2]:
# Create folders if they don't exist
os.makedirs('model', exist_ok=True)

# Generate synthetic data
X, y = make_regression(n_samples=100, n_features=2, noise=0.1, random_state=42)

# Train a basic model
model = LinearRegression()
model.fit(X, y)

# Save the model to disk
joblib.dump(model, 'model/model.pkl')

# Test that it works
print("Test prediction:", model.predict([[0.1, 0.2]]))

Test prediction: [23.58961171]


In [3]:
# Load the model
loaded_model = joblib.load('model/model.pkl')
print("Reloaded model prediction:", loaded_model.predict([[0.1, 0.2]]))

Reloaded model prediction: [23.58961171]


In [4]:
def predict(features, round_prediction=False):
    """
    Predict using the loaded model.
    Optional parameter: round_prediction
    """
    pred = loaded_model.predict([features])[0]
    if round_prediction:
        pred = round(pred, 2)
    return pred

# Test the function
print("Function test:", predict([0.1, 0.2], round_prediction=True))

Function test: 23.59


In [5]:
from flask import Flask, request, jsonify, render_template_string
import joblib
import matplotlib.pyplot as plt
import io, base64

model = joblib.load('model/model.pkl')
app = Flask(__name__)

def predict(features):
    pred = model.predict([features])[0]
    return pred

@app.route('/predict', methods=['POST'])
def predict_default():
    data = request.get_json()
    features = data.get('features')
    return jsonify({'prediction': float(predict(features))})

@app.route('/predict/<float:input1>')
@app.route('/predict/<float:input1>/<float:input2>')
def predict_path(input1, input2=None):
    if input2 is None:
        features = [input1]
    else:
        features = [input1, input2]
    return jsonify({'prediction': float(predict(features))})

@app.route('/plot')
def plot_page():
    fig, ax = plt.subplots()
    ax.plot([0,1,2,3],[10,20,15,30])
    ax.set_title("Demo Plot")
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    buf.close()
    html = f"<html><body><h1>Model Output Plot</h1><img src='data:image/png;base64,{img_base64}'></body></html>"
    return render_template_string(html)

if __name__ == '__main__':
    app.run(debug=False)

with open('app.py','w') as f:
    f.write(flask_code)

print("Flask app saved to app.py")

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


NameError: name 'flask_code' is not defined

In [7]:
import os
# Windows command to launch Flask in separate terminal window
os.system("start cmd /k python app.py")
print("Flask server launched in external terminal window.")

Flask server launched in external terminal window.


In [10]:
import streamlit as st
import joblib

model = joblib.load('model/model.pkl')

st.title("Model Prediction Dashboard")

input_1 = st.number_input("Feature 1")
input_2 = st.number_input("Feature 2")

if st.button("Predict"):
    pred = model.predict([[input_1, input_2]])
    st.write(f"Prediction: {pred[0]}")

with open('app_streamlit.py','w') as f:
    f.write(streamlit_code)

print("Streamlit app saved to app_streamlit.py")

# Launch Streamlit in external terminal (Windows example)
os.system("start cmd /k streamlit run app_streamlit.py")
print("Streamlit dashboard launched in external terminal window.")

2025-08-27 17:32:32.470 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 17:32:32.759 
  command:

    streamlit run E:\anaconda\envs\bootcamp_env\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-08-27 17:32:32.759 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 17:32:32.759 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 17:32:32.773 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 17:32:32.775 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 17:32:32.776 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 17:32:32.777 Thread 'MainThread

NameError: name 'streamlit_code' is not defined